# Motivation

Given a probability distribution $\pi$, the goal of MCMC is to simulate a random variable $X$ whose distribution is $\pi$.

The MCMC algorithm constructs an ergodic (irreducible, aperiodic, and all states have finite expected return times) Markov chain whose limiting distribution is the desired $\pi$.

We can run the chain long enough so that the chain converges, or nearly converges, to its limiting distribution. Then we can output the final element(s) of the Markov sequence as a sample from $\pi$.

# Law of Large Numbers

Let $Y_1, Y_2, \dots$ be an i.i.d. sequence with common mean $\mu < \infty$. The strong LLN states that, with probability 1,

$$\lim_{n\to\infty}\frac{Y_1 + \dots + Y_n}{n} = \mu.$$

Let $Y$ be a random variable with the same distribution as the $Y_i$ and assume $r$ is a bounded, real-valued function. Then, $r(Y_1), r(Y_2), \dots$ is also an i.i.d. sequence with finite mean, and, with probability 1,

$$\lim_{n\to\infty}\frac{r(Y_1) + \dots + r(Y_n)}{n} = E(r(Y)).$$

# Strong Law of Large Numbers for Markov Chain

Assume that $X_0, X_1, \dots$ is an ergodic Markov chain with stationary distribution $\pi$. Let $r$ be a bounded, real-valued function. Let $X$ be a random variable with distribution $\pi$.

Then, with probability 1,

$$\lim_{n\to\infty}\frac{r(X_1) + \dots + r(X_n)}{n} = E(r(X)),$$

where $E(r(X)) = \sum_{j} r(j)\pi_j$.



# Consequences of Strong LLN for Markov Chain

Given an ergodic Markov chain with stationary distribution $\pi$, let $A$ be a nonempty subset of the state space.

We can write $\pi_A = \sum_{j \in A}\pi_j$ and we can interpret $\pi_A$ as the long-term average number of visits to $A$ of an ergodic Markov chain.

Now, define the indicator variable

$$I_A(x) = \begin{cases} 1, &\mbox{if } x \in A, \\ 0, & \mbox{if } x \notin A. \end{cases}$$

Then, $\sum_{k = 0}^{n-1} I_A(X_k)$ is the number of visits to $A$ in the first $n$ steps of the chain.

Let X be a random variable with distribution $\pi$. With probability 1,

$$\lim_{n\to\infty}\frac{1}{n}\sum_{k = 0}^{n-1} I_A(X_k) = E(I_A(X)) = P(X \in A) = \pi_A.$$

# Example: Binary Sequences w/ No Adjacent 1s

Consider sequences of length $m$ consisting of 0s and 1s. A sequence is "good" if it has no adjacent 1s.

<b>Question:</b> What is the expected number of 1s in a good sequence if all good sequences are equally likely?

For general $m$, the expected number of 1s is $\mu = \sum_{k}k\pi_k$, where $\pi_k$ is the probability that a good sequence of length $m$ has exactly $k$ 1s.

There is no simple closed form expression for $\mu$. So, we use a simulation.

<b>The simulation:</b>

If $x$ is a sequence of 0s and 1s of length $m$, let $r(x)$ be the number of 1s in the sequence. Assume we can generate a uniformly random sequence of length $m$ with no adjacent 1s. Doing this repeatedly and independently, generating good sequences $Y_1, Y_2, \dots, Y_n$, we can generate a Monte Carlo estimate of the desired expectation with

$$\mu \approx \frac{r(Y_1) + r(Y_2) + \dots + r(Y_n)}{n}$$

for large $n$. This is by the LLN.

Thus, the problem of estimating $\mu$ reduces itself to the problem of simulating a good sequence.

<b>Generating a good sequence:</b>

The idea is to construct an ergodic Markov chain $X_0, X_1, \dots$ whose state space is the set of good sequences and whose limiting distribution is uniform on the set of good sequences.

The Markov chain is then generated and, as in the i.i.d. case, we take

$$\mu \approx \frac{r(X_1) + r(X_2) + \dots + r(X_n)}{n}$$

for large $n$, as an MCMC estimate for the desired expectation.

<b>Constructing the Markov chain:</b>

The Markov chain is constructed as a random walk on a graph whose vertices are good sequences. Proceed as follows.

From a given good sequence, pick one of its $m$ components uniformly at random.

If the component is 1, then switch it to 0. The new sequence is also a good sequence. Move to the new sequence.

If the component is 0, then switch it to 1 only if the resulting sequence is good. If it is, then move to the new sequence. Otherwise, stay put and the walk stays at the current state.

<b>Implementing the random walk algorithm:</b>

Starting with the sequence of all 0s as the initial state of the chain...

In [13]:
import numpy as np

m = 100
n = 100000
seq = [0 for x in range(m)]
chainSeq = [seq]
mu = 0
for i in range(n):
    randInt = np.random.randint(0,m)
    if seq[randInt] == 1:
        seq[randInt] = 0
    else:
        if randInt == 0:
            if seq[randInt+1] == 0:
                seq[randInt] = 1
        elif randInt == (m-1):
            if seq[randInt-1] == 0:
                seq[randInt] = 1
        else:
            if seq[randInt+1] == 0 and seq[randInt-1] == 0:
                seq[randInt] = 1
    chainSeq.append(seq)
    mu += sum(seq)
mu /= n
print(mu)

27.90368


The above value is the MCMC estimate for the expected number of 1s, $\mu$. An exact analysis for $m = 100$ gives $\mu = 27.7921$.

Note that the uniform distribution on the set of good sequences assigns probability $\frac{1}{c}$ to each sequence, where $c$ is the number of good sequences. The actual value of $c$ was never needed in this implementation and could have even been unknown.

# Metropolis-Hastings Algorithm

Let $\pi = (\pi_1, \pi_2, \dots)$ be a discrete probability distribution. The algorithm constructs a reversible Markov chain $X_0, X_1, \dots$ whose stationary distribution is $\pi$.

Let $T$ be a transition matrix for any irreducible Markov chain with the same state space as $\pi$. We assume we know how to sample from $T$. The $T$ chain will be used as a proposal chain, generating elements of a sequence that the algorithm decides whether or not to accept.

To describe the transition mechanism for $X_0, X_1, \dots$, assume that at time $n$ the chain is at state $i$. The next step of the chain $X_{n+1}$ is determined by a two-step procedure:

1. Choose a new state according to $T$. That is, choose $j$ with probability $T_{ij}$. State $j$ is called the proposal state.

2. Decide whether or not to accept $j$ or not. Let

    $$a(i,j) = \frac{\pi_j T_{ji}}{\pi_i T_{ij}}.$$

    The function $a$ is called the acceptance function. If $a(i,j) \geq 1$, then $j$ is accepted at the state of the chain. If $a(i,j) < 1$, then $j$ is accepted with probability $a(i,j)$. If $j$ is not accepted, then $i$ is kept at the next state of the chain.

In other words, assume $X_n = i$. Let $U$ be uniformly distributed on $(0,1)$. Then, set

$$X_{n+1}(x) = \begin{cases} j, &\mbox{if } U \leq a(i,j), \\ i, & \mbox{if } U > a(i,j). \end{cases}$$

The sequence $X_0, X_1, \dots$ constructed by th Metropolis-Hastings algorithm is a reversible Markov chain whose stationary distribution is $\pi$.

<b>Note:</b>

1. The exact form of $\pi$ is not necessary to implement this algorithm. It only uses ratios of the form $\pi_j/\pi_i$ and thus $\pi$ only needs to be specified up to proportionality. If $\pi$ is uniform on a set of size $c$, then $\pi_j/\pi_i = 1$ and the acceptance function reduces to $a(i,j) = T_{ji}/T_{ij}$.

2. If the proposal transition matrix $T$ is symmetric, then $a(i,j) = \pi_j/\pi_i$.

3. The algorithm works for any irreducible proposal chain.

4. If the proposal chain is ergodic, then the resulting Metropolis-Hastings chain is also ergodic with limiting distribution $\pi$.

5. The generated sequence $X_0, X_1, \dots$, gives an approximate sample from $\pi$. However, if the chain requires many steps to get close to stationary, there may be initial bias. Burn-in refers to the practice of discarding the initial iterations and retaining $X_m, X_{m+1}, \dots, X_n$ for some $m$. In that case, the strong LLN for Markov chains gives

$$\lim_{n\to\infty}\frac{r(X_m) + \dots + r(X_n)}{n-m+1} = \sum_{x} r(x)\pi_x.$$

# Metropolis-Hastings for Continuous Case

MCMC can also be used in the continuous case, when $\pi$ is a probability density function.

For the continuous case, the transition matrix is replaced by a transition function, where $P_{ij}$ is the value of a conditional density function given $X_0 = i$.

The algorithm is essentially the same as in the discrete case, with densities replacing discrete distributions.